## Weight ablations

In this ablation study we verify how much does the performance of epsilon metric change depending on the chosen weights.

In [1]:
import os
import json
import time
import itertools

import numpy as np
import pandas as pd
import pickle as pkl
import seaborn as sns
from scipy import stats
from tqdm import trange
from dotmap import DotMap
from statistics import mean
import matplotlib.pyplot as plt
from prettytable import PrettyTable

import torch.nn as nn
import torch

import nasspace
from datasets import data
from epsilon_utils import prepare_seed, compute_stats, plot_results

2022-09-29 18:03:07.409761: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2022-09-29 18:03:07.409882: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64:/usr/local/cuda-10.1/lib64:/usr/local/cuda-10.1/extras/CUPTI/lib64
2022-09-29 18:03:07.409894: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above

In [5]:
import pickle
import os

import matplotlib.pyplot as plt
import numpy as np


In [2]:
args = DotMap()
args.data_loc = '../cifardata'
args.api_loc = './nasbench_only108.tfrecord'
args.save_loc = './results'
args.score = 'hook_logdet'
args.nasspace = 'nasbench101'
args.batch_size=256
args.repeat=1
args.sigma=0.05
args.GPU='3'
args.seed=1
args.init=''
args.dataset='cifar10'
args.maxofn=1
args.n_samples=100
args.n_runs=5
args.stem_out_channels = 128
args.num_stacks = 3
args.num_modules_per_stack = 3
args.num_labels = 1
args.augtype='none'
args.trainval=False

In [3]:
# savedataset = args.dataset
# dataset = 'fake' if 'fake' in savedataset else savedataset
# savedataset = savedataset.replace('fake', '')
# if savedataset == 'cifar10':
#     savedataset = savedataset + '-valid'
# searchspace = nasspace.get_search_space(args)
# if 'valid' in savedataset:
#     savedataset = savedataset.replace('-valid', '')
    
# if args.dataset == 'cifar10':
#     acc_type = 'ori-test'
#     val_acc_type = 'x-valid'
# else:
#     acc_type = 'x-test'
#     val_acc_type = 'x-valid'

Loading dataset from file... This may take a few minutes...
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded dataset in 73 seconds


In [6]:
synflow_proxy=[]
save_dir = '/home/gracheva/Work/NAS/NAS-Bench-101/release_results/NAS-Bench-101/CIFAR10/WEIGHT_0.0001_10/BS_256'

f = open('{}/Data'.format(save_dir),'rb')
while(1):
    try:
        d = pickle.load(f)
        nparams = d['nparams']
        synflow_proxy = d['score']

    except EOFError:
        break
f.close()

# Treat NaNs
synflow_proxy = np.array(synflow_proxy)
synflow_proxy[np.isnan(synflow_proxy)]=0

In [12]:
#dicts to map from index to spec or spec to index
#spec is the model description within NAS, in our case we use an encoding [5, 5, 5, 5, 5, 5]
#this is a length-6 vector, with each entry having a value between 0-4 
#this is sufficient to describe any NAS-Bench-201 cell

_opname_to_index = {
    'none': 0,
    'skip_connect': 1,
    'nor_conv_1x1': 2,
    'nor_conv_3x3': 3,
    'avg_pool_3x3': 4
}

def get_spec_from_arch_str(arch_str):
    print(arch_str)
    nodes = arch_str.split('+')
    nodes = [node[1:-1].split('|') for node in nodes]
    nodes = [[op_and_input.split('~')[0]  for op_and_input in node] for node in nodes]

    spec = [_opname_to_index[op] for node in nodes for op in node]
    return spec

idx_to_spec = {}
for i, arch_str in enumerate(searchspace.mutate_arch):
    idx_to_spec[i] = get_spec_from_arch_str(arch_str)

spec_to_idx = {}
for idx,spec in idx_to_spec.items():
    spec_to_idx[str(spec)] = idx

TypeError: 'method' object is not iterable

In [ ]:
import random
import copy

def random_spec():
    return random.choice(list(idx_to_spec.values()))

def mutate_spec(old_spec):
    idx_to_change = random.randrange(len(old_spec))
    entry_to_change = old_spec[idx_to_change]
    possible_entries = [x for x in range(5) if x != entry_to_change]
    new_entry = random.choice(possible_entries)
    new_spec = copy.copy(old_spec)
    new_spec[idx_to_change] = new_entry
    return new_spec

In [ ]:
def random_combination(iterable, sample_size):
    pool = tuple(iterable)
    n = len(pool)
    indices = sorted(random.sample(range(n), sample_size))
    return tuple(pool[i] for i in indices)

In [ ]:
def run_evolution_search(max_trained_models=1000, 
                         pool_size=64, 
                         tournament_size=10, 
                         zero_cost_warmup=0, 
                         zero_cost_move=False):
    
    best_valids, best_tests = [0.0], [0.0]
    pool = []   # (validation, spec) tuples
    num_trained_models = 0

    # fill the initial pool
    if zero_cost_warmup > 0:
        zero_cost_pool = []
        for _ in range(zero_cost_warmup):
            spec = random_spec()
            spec_idx = spec_to_idx[str(spec)]
            zero_cost_pool.append((synflow_proxy[spec_idx], spec))
            zero_cost_pool = sorted(zero_cost_pool, key=lambda i:i[0], reverse=True)
    for i in range(pool_size):
        if zero_cost_warmup > 0:
            spec = zero_cost_pool[i][1]
        else:
            spec = random_spec()
#         info = api.get_more_info(spec_to_idx[str(spec)], 'ImageNet16-120', iepoch=None, hp='200', is_random=False)
        info = api.get_more_info(spec_to_idx[str(spec)], 'cifar100', iepoch=None, hp='200', is_random=False)

        num_trained_models += 1
        pool.append((info['valid-accuracy'], spec))

        if info['valid-accuracy'] > best_valids[-1]:
            best_valids.append(info['valid-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            
        if info['test-accuracy'] > best_tests[-1]:
            best_tests.append(info['test-accuracy'])
        else:
            best_tests.append(best_tests[-1])

    # After the pool is seeded, proceed with evolving the population.
    while(1):
        if zero_cost_move:
            parent = sorted(pool, key=lambda i:i[0])[-1][1]
        else:
            parent = random_combination(pool, 1)[0][1]
        
        child = mutate_spec(parent)

        info = api.get_more_info(spec_to_idx[str(child)], 'cifar100', iepoch=None, hp='200', is_random=False)
        num_trained_models += 1

        # kill the oldest individual in the population.
        if zero_cost_move:
            pool.append((synflow_proxy[spec_to_idx[str(child)]], child))
        else:
            pool.append((info['valid-accuracy'], child))
        pool.pop(0)

        if info['valid-accuracy'] > best_valids[-1]:
            best_valids.append(info['valid-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            
        if info['test-accuracy'] > best_tests[-1]:
            best_tests.append(info['test-accuracy'])
        else:
            best_tests.append(best_tests[-1])

        if num_trained_models >= max_trained_models:
            break
    best_tests.pop(0)
    best_valids.pop(0)
    return best_valids, best_tests

In [ ]:
def run_evolution_search(max_trained_models=1000, 
                         pool_size=64, 
                         tournament_size=10, 
                         zero_cost_warmup=0, 
                         zero_cost_move=False):
    
    best_valids, best_tests = [0.0], [0.0]
    pool = []   # (validation, spec) tuples
    num_trained_models = 0

    # fill the initial pool
    if zero_cost_warmup > 0:
        zero_cost_pool = []
        for _ in range(zero_cost_warmup):
            spec = random_spec()
            spec_idx = spec_to_idx[str(spec)]
            zero_cost_pool.append((synflow_proxy[spec_idx], spec))
            zero_cost_pool = sorted(zero_cost_pool, key=lambda i:i[0], reverse=True)
    for i in range(pool_size):
        if zero_cost_warmup > 0:
            spec = zero_cost_pool[i][1]
        else:
            spec = random_spec()
#         info = api.get_more_info(spec_to_idx[str(spec)], 'ImageNet16-120', iepoch=None, hp='200', is_random=False)
        info = api.get_more_info(spec_to_idx[str(spec)], 'cifar100', iepoch=None, hp='200', is_random=False)

        num_trained_models += 1
        pool.append((info['valid-accuracy'], spec))

        if info['valid-accuracy'] > best_valids[-1]:
            best_valids.append(info['valid-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            
        if info['test-accuracy'] > best_tests[-1]:
            best_tests.append(info['test-accuracy'])
        else:
            best_tests.append(best_tests[-1])

    # After the pool is seeded, proceed with evolving the population.
    while(1):
        if zero_cost_move:
            parent = sorted(pool, key=lambda i:i[0])[-1][1]
        else:
            parent = random_combination(pool, 1)[0][1]
        
        child = mutate_spec(parent)

        info = api.get_more_info(spec_to_idx[str(child)], 'cifar100', iepoch=None, hp='200', is_random=False)
        num_trained_models += 1

        # kill the oldest individual in the population.
        if zero_cost_move:
            pool.append((synflow_proxy[spec_to_idx[str(child)]], child))
        else:
            pool.append((info['valid-accuracy'], child))
        pool.pop(0)

        if info['valid-accuracy'] > best_valids[-1]:
            best_valids.append(info['valid-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            
        if info['test-accuracy'] > best_tests[-1]:
            best_tests.append(info['test-accuracy'])
        else:
            best_tests.append(best_tests[-1])

        if num_trained_models >= max_trained_models:
            break
    best_tests.pop(0)
    best_valids.pop(0)
    return best_valids, best_tests

In [ ]:
def run_random_search(max_trained_models=1000, 
                      zero_cost_warmup=0):
    
    best_valids, best_tests = [0.0], [0.0]
    pool = []   # (validation, spec) tuples
    num_trained_models = 0

    # fill the initial pool
    if zero_cost_warmup > 0:
        zero_cost_pool = []
        for _ in range(zero_cost_warmup):
            spec = random_spec()
            spec_idx = spec_to_idx[str(spec)]
            zero_cost_pool.append((synflow_proxy[spec_idx], spec))
            zero_cost_pool = sorted(zero_cost_pool, key=lambda i:i[0], reverse=True)
    for i in range(max_trained_models):
        if i < zero_cost_warmup:
            spec = zero_cost_pool[i][1]
        else:
            spec = random_spec()
        info = api.get_more_info(spec_to_idx[str(spec)], 'cifar100', iepoch=None, hp='200', is_random=False)

        if info['valid-accuracy'] > best_valids[-1]:
            best_valids.append(info['valid-accuracy'])
        else:
            best_valids.append(best_valids[-1])
            
        if info['test-accuracy'] > best_tests[-1]:
            best_tests.append(info['test-accuracy'])
        else:
            best_tests.append(best_tests[-1])
            
    best_tests.pop(0)
    best_valids.pop(0)
    return best_valids, best_tests

In [ ]:
from tqdm import tqdm
random.seed = 21
num_rounds = 100
length = 300
ae, ae_warmup, ae_move, ae_warmup_move, rand, rand_warmup = [], [], [], [], [], []
for _ in tqdm(range(num_rounds)):
    ae_best_valids, ae_best_tests = run_evolution_search(max_trained_models=length)
    ae.append(ae_best_tests)
    ae_warmup_best_valids, ae_warmup_best_tests = run_evolution_search(max_trained_models=length, zero_cost_warmup=3000)
    ae_warmup.append(ae_warmup_best_tests)
    ae_move_best_valids, ae_move_best_tests = run_evolution_search(max_trained_models=length, zero_cost_move=True)
    ae_move.append(ae_move_best_tests)
    ae_warmup_move_best_valids, ae_warmup_move_best_tests = run_evolution_search(max_trained_models=length, zero_cost_warmup=3000, zero_cost_move=True)
    ae_warmup_move.append(ae_warmup_move_best_tests)
    rand_best_valids, rand_best_tests = run_random_search(max_trained_models=length)
    rand.append(rand_best_tests)
    rand_warmup_best_valids, rand_warmup_best_tests = run_random_search(max_trained_models=length, zero_cost_warmup=3000)
    rand_warmup.append(rand_warmup_best_tests)

In [ ]:
def plot_experiment(exp_list, title, filename):
    fig = plt.figure(figsize=(6,4.5))
    ax = fig.add_subplot(111)
    def plot_exp(exp, label):
        exp = np.array(exp) 
        q_75 = np.quantile(exp, .75, axis=0)
        q_25 = np.quantile(exp, .25, axis=0)
        median = np.median(exp, axis=0)
        ax.plot(median, label=label)
        ax.fill_between(range(len(q_25)), q_25, q_75, alpha=0.1)
    for exp,ename in exp_list:
        plot_exp(exp, ename)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(16)
    
    plt.grid()
    plt.xlabel('Trained Models', fontsize = 22)
    plt.ylabel('Test Accuracy', fontsize = 22)
    plt.ylim(70,73.6) #89,92   44,47    70,73.6
    plt.legend(fontsize = 16)
    plt.title(title, fontsize = 22)
    plt.savefig(filename,
                bbox_inches='tight', 
                dpi=300,
                format='pdf')
    plt.show()

In [ ]:
save_dir = '/home/gracheva/Work/NAS/NAS-Bench-101/release_results/NAS-Bench-101/RE'
os.makedirs(save_dir, exist_ok=True)
filename=save_dir + 'Evolution_Len300_Rounds100_Move_RE.pdf'
plot_experiment([(ae,'AE'), (ae_warmup,'AE + warmup'), (ae_move,'AE + move'), (ae_warmup_move, 'AE + warmup + move')], 'Aging Evolution Search', filename)

In [ ]:
filename = save_dir+"_RS3000+RS_100it.pdf"
plot_experiment([(rand,'RAND'), (rand_warmup,'RAND + warmup (3000)')], 'Random Search', filename)